Для распознавания используется предварительно обученная сверточная нейронная сеть VGG16.

Перед использованием необходимо скачать и подготовить данные для обучения, проверки и тестирования.

In [0]:
#from google.colab import drive
#drive.mount('/content/gdrive')

In [0]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.python.keras.applications import VGG16
from tensorflow.python.keras.optimizers import Adam

from tensorflow.python.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.python.keras.layers.normalization import BatchNormalization

from tensorflow.python.keras.applications import InceptionV3

In [0]:
# Каталог с данными для обучения
train_dir = '/content/gdrive/My Drive/DataSet/test'
# Каталог с данными для проверки
val_dir = '/content/gdrive/My Drive/DataSet/val'
# Каталог с данными для тестирования
test_dir = '/content/gdrive/My Drive/DataSet/train'
# Размеры изображения
img_width, img_height = 150, 150
# Размерность тензора на основе изображения для входных данных в нейронную сеть
# backend Tensorflow, channels_last
input_shape = (img_width, img_height, 3)
# Размер мини-выборки
batch_size = 32
# Количество изображений для обучения
nb_train_samples = 2565
# Количество изображений для проверки
nb_validation_samples = 550
# Количество изображений для тестирования
nb_test_samples = 550

Загружаем предварительно обученную нейронную сеть

In [0]:
vgg16_net = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
# Загружаем предварительно обученную нейронную сеть Inceptionv3
#inception_net = InceptionV3(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

Создаем составную нейронную сеть на основе VGG16 или inception

In [25]:
vgg16_net.summary()
#inception_net.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [0]:
model = Sequential()
# Добавляем в модель сеть VGG16 вместо слоя
model.add(vgg16_net)
#model.add(inception_net)
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='sigmoid'))

Компилируем составную нейронную сеть

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=1e-5), 
              metrics=['accuracy'])

Создаем генератор изображений
Генератор изображений создается на основе класса ImageDataGenerator. Генератор делит значения всех пикселов изображения на 255.

In [0]:
datagen = ImageDataGenerator(rescale=1./255)

Генератор данных для обучения на основе изображений из каталога

In [29]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 126 images belonging to 6 classes.


Генератор данных для проверки на основе изображений из каталога

In [30]:
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 0 images belonging to 0 classes.


Генератор данных для тестирования на основе изображений из каталога

In [31]:
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 336 images belonging to 6 classes.


Обучаем модель с использованием генераторов
train_generator - генератор данных для обучения

validation_data - генератор данных для проверки

In [32]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=10,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/10


W0617 02:26:34.897899 140510202988416 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


79/80 [============================>.] - ETA: 0s - loss: 0.9180 - acc: 0.6768

KeyboardInterrupt: ignored

In [0]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)

In [0]:
print("Аккуратность на тестовых данных: %.2f%%" % (scores[1]*100))

image_file_name = train_dir + '/buildings/0.jpg'
img = image.load_img(image_file_name, target_size=(150, 150))
plt.imshow(img)
#plt.show()

xxxxx = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)

xxxxx = xxxxx.reshape((1,) + xxxxx.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

xxxxx = preprocess_input(xxxxx)
preds = model.predict(xxxxx)
print('Predicted:', preds)
	
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
print("Аккуратность на тестовых данных: %.2f%%" % (scores[1]*100))

model.save_weights(name)
print(name + " saved model to disk")
